# *cpdctl* Sample Code for Promoting Scripts to a Deployment Space

## Before you begin
Import the following libraries:

In [ ]:
# Import required libraries and modules
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
import jmespath
import subprocess
from IPython.core.display import display, HTML

##  Installing and configure CPDCTL <a class="anchor" id="part1"></a>

### Install the latest version of CPDCTL <a class="anchor" id="part1.1"></a>### Install the version v1.1.132 of `cpdctl`

To use the notebook and environment CLI commands, you need to install CPDCTL. Download the binary from the [CPDCTL GitHub respository](https://github.com/IBM/cpdctl/releases).

Download the binary and then display the version number:

In [ ]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')
version='v1.1.132'

response = requests.get(CPDCTL_RELEASES_URL)
asset_version = next(a for a in response.json() if version==a['tag_name'])
#assets = response.json()[0]['assets']
assets=asset_version['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [ ]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [ ]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

### Add CPD cluster configuration settings <a class="anchor" id="part1.2"></a>

Before you can use CPDCTL, you need to add configuration settings. You only need to configure these settings once for the same IBM Cloud Pak for Data (CPD) user and cluster. Begin by entering your CPD credentials and the URL to the CPD cluster:

In [ ]:
# These values are needed only when using cpdctl outside of CPD
#CPD_USERNAME = ' ' # for example: datascientist
#CPD_PASSWORD = ' '
#CPD_URL = ' ' #typically, this would be https://cpd-cpd-instance.apps.demo.ibmdte.net

List available spaces in context:

In [ ]:
! cpdctl space list

Choose the space to which you will copy the script

In [ ]:
#result = ! cpdctl space list --output json -j "(resources[].metadata.id)[0]" --raw-output
#space_id = result.s
# print("space id: {}".format(space_id))

# You can also specify your space id directly:
space_id = " "

## 2.Creating a script asset and in the deployment space and run a job <a class="anchor" id="part2"></a>

### 2.1 Create a script asset<a class="anchor" id="part2.1"></a>

First, we create a *script asset* in your project. Assets are used to capture various metadata. To create a script asset you need to specify various fields as showin in metadata and location of the script file in JupyterLab file directory.  

In our example we created a script directory on the same level as *notebooks* and *data_assets* directories that are created by default in Watson Studio. We also created a simple script that we named *testscript.py*

In [ ]:
# Display the current directory so that we can correctly specify the directory to the /script folder
!pwd

In [ ]:

# for the Remote path (the path in the Deployment space), we must always use the script/prefix
remote_file_path = "script/testscript.py"
# Local path may be different in your en
local_file_path = "../script/testscript.py"

In [ ]:
# Upload the file to the temp space used by cpdctl for copying files between projects and spaces
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --space-id {space_id}

Create a script asset:

In [ ]:
# Specify the metadata, entity and attachments of the script file:

metadata = {
    "name": "my_test_script_with_env_var",
    "asset_type": "script",
    "asset_category": "USER",
    "origin_country": "us"
}
metadata_json = json.dumps(metadata)

entity = {
    "script": {
        "language": {
            "name": "python3"
        }
    }
}
entity_json = json.dumps(entity)

attachments = [
    {
        "asset_type": "script",
        "name": "my_test_script",
        "description": "attachment for script",
        "mime": "application/text",
        "object_key": remote_file_path
    }
]
attachments_json = json.dumps(attachments)

In [ ]:
# Create a Python script asset in the deployment space:

result = ! cpdctl asset create  --metadata '{metadata_json}' --entity '{entity_json}' --attachments '{attachments_json}' --space-id {space_id} --output json -j "metadata.asset_id" --raw-output
script_id = result.s
print("script id: {}".format(script_id))

<span style="color:red">Important Note: Check the target deployment space. *Notebook1* should show up in the Assets tab. </span>

Copyright © 2021 IBM. This notebook and its source code are released under the terms of the MIT License.